In [1]:
!pip install numpy

                                              0.0/15.8 MB ? eta -:--:--
     -                                        0.5/15.8 MB 11.1 MB/s eta 0:00:02
     -----                                    2.2/15.8 MB 24.0 MB/s eta 0:00:01
     ---------                                3.6/15.8 MB 25.4 MB/s eta 0:00:01
     ------------                             5.1/15.8 MB 29.7 MB/s eta 0:00:01
     ----------------                         6.6/15.8 MB 28.1 MB/s eta 0:00:01
     -------------------                      7.6/15.8 MB 30.3 MB/s eta 0:00:01
     -------------------                      7.6/15.8 MB 30.3 MB/s eta 0:00:01
     -------------------------               10.2/15.8 MB 27.1 MB/s eta 0:00:01
     -----------------------------           12.1/15.8 MB 29.7 MB/s eta 0:00:01
     ----------------------------------      14.0/15.8 MB 31.2 MB/s eta 0:00:01
     --------------------------------------  15.8/15.8 MB 34.4 MB/s eta 0:00:01
     --------------------------------------- 15.


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install torch

     ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
     ---------------------------------------- 0.3/172.3 MB 5.7 MB/s eta 0:00:31
     ---------------------------------------- 0.6/172.3 MB 6.8 MB/s eta 0:00:26
     ---------------------------------------- 1.1/172.3 MB 7.6 MB/s eta 0:00:23
     ---------------------------------------- 1.6/172.3 MB 8.6 MB/s eta 0:00:20
      --------------------------------------- 2.3/172.3 MB 9.8 MB/s eta 0:00:18
      -------------------------------------- 3.0/172.3 MB 10.6 MB/s eta 0:00:17
      -------------------------------------- 3.9/172.3 MB 11.8 MB/s eta 0:00:15
     - ------------------------------------- 5.0/172.3 MB 13.3 MB/s eta 0:00:13
     - ------------------------------------- 6.3/172.3 MB 14.9 MB/s eta 0:00:12
     - ------------------------------------- 7.9/172.3 MB 16.8 MB/s eta 0:00:10
     -- ------------------------------------ 9.8/172.3 MB 18.9 MB/s eta 0:00:09
     -- ----------------------------------- 11.

In [1]:
!pip install onnxruntime-gpu

  Obtaining dependency information for onnxruntime-gpu from https://files.pythonhosted.org/packages/5a/9d/19367cf0de7b9b01678b3e4279a59b472a09f11bb2ab79eee62e1916b927/onnxruntime_gpu-1.16.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/46.0 kB ? eta -:--:--
     ---------------------------------------- 46.0/46.0 kB 2.4 MB/s eta 0:00:00
  Obtaining dependency information for flatbuffers from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Obtaining dependency information for protobuf from https://files.pythonhosted.org/packages/5e/46/5b9674a33cbf690ffdd79ab1863767a66461cd06ea7aeb9f90e4e50be7a5/protobuf-4.24.3-cp310-abi3-win_amd64.whl.metadata
     ---------------------------------------- 0.0/86.8 kB ? eta -:--:--
     ---------------------------------------- 86.8/86.8 kB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 0.

In [15]:
!pip install onnx

  Obtaining dependency information for onnx from https://files.pythonhosted.org/packages/03/49/7263b3806ffebd3c967341986df32a5e62b2fa78beca2cdf9516d876b3fc/onnx-1.14.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.3 MB 15.5 MB/s eta 0:00:01
   ------ --------------------------------- 2.1/13.3 MB 26.3 MB/s eta 0:00:01
   --------- ------------------------------ 3.0/13.3 MB 24.2 MB/s eta 0:00:01
   ------------ --------------------------- 4.0/13.3 MB 25.6 MB/s eta 0:00:01
   --------------- ------------------------ 5.1/13.3 MB 25.1 MB/s eta 0:00:01
   ------------------- -------------------- 6.5/13.3 MB 27.7 MB/s eta 0:00:01
   ------------------------ --------------- 8.3/13.3 MB 29.4 MB/s eta 0:00:01
   ----------------------------- ---------- 9.9/13.3 MB 31.4 MB/s eta 0:00:01
   ---------------------------------- ----- 11.5/13.3 MB 34.6 MB/s eta 0:00:01
   ---------------------

In [8]:
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx


In [9]:
import torch.nn as nn
import torch.nn.init as init

class SuperResolutionNet(nn.Module):
    def __init__(self, upsace_factor, inplace=False):
        super(SuperResolutionNet,self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1,64,(5,5),(1,1),(2,2))
        self.conv2 = nn.Conv2d(64, 64, (3,3), (1,1), (1,1))
        self.conv3 = nn.Conv2d(64, 32, (3,3), (1,1), (1,1))
        self.conv4 = nn.Conv2d(32, upsace_factor**2, (3,3),(1,1), (1,1))
        self.pixel_shuffle = nn.PixelShuffle(upsace_factor)
        self._initialize_weights()


    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

In [10]:
torch_model = SuperResolutionNet(upsace_factor=3)

In [11]:
# Load pretrained model weights
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 1    # just a random number

# Initialize model with the pretrained weights
map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None
torch_model.load_state_dict(model_zoo.load_url(model_url, map_location=map_location))

# set the model to inference mode
torch_model.eval()

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pixel_shuffle): PixelShuffle(upscale_factor=3)
)

In [16]:
# Input to the model
import torch.onnx
import onnx
batch_size = 1
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
torch_out = torch_model(x)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

